In [1]:
import ROOT as r
from bff_processor.Systogram import Systogram
import pickle
import re
import uncertainties
import numpy as np

Welcome to JupyROOT 6.14/04


In [28]:
era = '2016'
if era=='2016':
    lumi=1.025
if era=='2017':
    lumi=1.023
if era=='2018':
    lumi=1.025

In [29]:
outname="fits/{}_functional_plotter.pkl".format(era)
with open(outname,'rb') as f:
    processed_stuff= pickle.load(f)

In [30]:
def val2root(edges, values, std, name):
    nBins = len(edges) - 1
    th1f =  r.TH1F(name, name, nBins, edges[0], edges[-1])
    for i in range(th1f.GetNbinsX()):
        th1f.SetBinContent(i, values[i])
        th1f.SetBinError(i, std[i]**.5)
    return th1f

In [31]:
def hist2DH(hist, name):
    values = hist.values()
    variances = hist.variances()
    edges = hist.axes[0].edges
    hist = val2root(edges,values,variances**.5, name)
    return r.RooDataHist(name, name, r.RooArgList(x), hist)

In [32]:
x = r.RooRealVar('x', 'x', 110, 800)

In [33]:
hist = processed_stuff['bff_list'][0]['SR1']
dh_signal_hists = []
for reg in processed_stuff['bff_list']:
    #print(reg)
    for sr in ['SR1', 'SR2']:
        name = re.findall(r'(BFF.+).csv',reg['data'][0])[0]
        name = sr+'_'+name
        dh_signal_hists.append(hist2DH(reg[sr], name))
        edges = reg[sr].axes[0].edges
        for hist_name, up, down in reg[sr].systematics:
            values, hists = list(zip(*sorted(zip([np.sum(up.values()),np.sum(down.values())], [up,down]))))
            dh_signal_hists.append(hist2DH(hists[1], name+'_'+hist_name+'Up'))
            dh_signal_hists.append(hist2DH(hists[0], name+'_'+hist_name+'Down'))

[#1] INFO:DataHandling -- RooDataHist::adjustBinning(SR1_BFFZprimeToMuMu_M_500_dbs0p04): fit range of variable x expanded to nearest bin boundaries: [110,800] --> [110,800]
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuUp): fit range of variable x expanded to nearest bin boundaries: [110,800] --> [110,800]
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuDown): fit range of variable x expanded to nearest bin boundaries: [110,800] --> [110,800]
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagUp): fit range of variable x expanded to nearest bin boundaries: [110,800] --> [110,800]
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagDown): fit range of variable x expanded to nearest bin boundaries: [110,800] --> [110,800]
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(SR1_BFFZprimeToMuMu_M_500_

Warning in <TROOT::Append>: Replacing existing TH1: SR1_BFFZprimeToMuMu_M_500_dbs0p04 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuUp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuDown (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagUp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagDown (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDUp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDDown (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PDF_ISRFSR_Up (Potential memory leak).
Warni

In [34]:
import glob
files = [f for f in glob.glob('fits/*') if (era in f) and ('pkl' in f) and ('data' in f)]

In [35]:
abcd_pred = []
for file in files:
    with open(file, 'rb') as f:
        abcd_dict = pickle.load(f)
    nJets = re.findall(r'/(\d)_',file)[0]
    print(nJets)
    val_std = [(x.nominal_value, x.std_dev) for x in abcd_dict]
    value, stdev = list(zip(*val_std))
    hist = val2root(edges, value, stdev, "background_{}".format(nJets))
    abcd_pred.append(r.RooDataHist("SR{}_background".format(nJets), "SR{}_background".format(nJets), r.RooArgList(x),hist))

1
2
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(SR1_background): fit range of variable x expanded to nearest bin boundaries: [110,800] --> [110,800]
[#1] INFO:DataHandling -- RooDataHist::adjustBinning(SR2_background): fit range of variable x expanded to nearest bin boundaries: [110,800] --> [110,800]


In [36]:
w = r.RooWorkspace("zp".format(era), "zp".format(era))

# Import abcd models
for dh in dh_signal_hists:
    print(dh)
    getattr(w, 'import')(dh)
    #break
    
for dh in abcd_pred:
    print(dh)
    getattr(w, 'import')(dh)
    #break
    
getattr(w, 'import')(x)

# S a v e   w o r k s p a c e   i n   f i l e
# -------------------------------------------

# Save the workspace into a ROOT file
w.writeToFile("era_{}_zp_ws.root".format(era))



<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04") at 0x7dd59a0>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuUp") at 0x73a9900>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuDown") at 0x71cbd90>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagUp") at 0x82a8be0>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagDown") at 0x7c06b60>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDUp") at 0x715c460>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDDown") at 0x714b290>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PDF_ISRFSR_Up") at 0x7d826b0>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PDF_ISRFSR_Down") at 0x7960db0>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_MuonSFUp") at 0x72430d0>
<ROOT.RooDataHist object ("SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_MuonSFDown") at 

False

[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_500_dbs0p04
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuUp
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuDown
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagUp
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagDown
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDUp
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDDown
[#1] INFO:ObjectHandling -- RooWorkspace::import(zp) importing dataset SR1_BFFZprimeToMuMu_M_500_dbs0p

In [37]:
w.Print()


RooWorkspace(zp) zp contents

variables
---------
(x)

datasets
--------
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PuDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_BTagDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PUIDDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PDF_ISRFSR_Up(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_PDF_ISRFSR_Down(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_MuonSFUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_MuonSFDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_ElectronSFUp(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_Weight_ElectronSFDown(x)
RooDataHist::SR1_BFFZprimeToMuMu_M_500_dbs0p04_jerUp(x)
RooDataHist::S

In [38]:
from combine_templates.combine_template import combine_template

In [39]:
for dh in dh_signal_hists:
    name = dh.GetName()
    nJets = re.findall(r'SR(\d)', name)[0]
    mass = re.findall(r'M_(\d+)_', name)[0]
    dbs = re.findall(r'dbs(\d)p(\d+)', name)[0]
    dbs = '{}.{}'.format(*dbs)
    sys = len(re.findall(r'dbs(\d)p(\d)+_', name)) > 0
    if sys: continue
    
    format_dict = {'era': era, 'mass': mass, 'nJets': nJets, 'dbs':dbs, 'lumi':lumi, 'signame':name}
    
    print(nJets, mass, dbs)
    print(name)
    with open('combine_cfgs/{}_{}.txt'.format(era,name), 'w') as f:
        print(combine_template.format(**format_dict), file=f)

1 500 0.04
SR1_BFFZprimeToMuMu_M_500_dbs0p04
2 500 0.04
SR2_BFFZprimeToMuMu_M_500_dbs0p04
1 250 0.04
SR1_BFFZprimeToMuMu_M_250_dbs0p04
2 250 0.04
SR2_BFFZprimeToMuMu_M_250_dbs0p04
1 200 0.04
SR1_BFFZprimeToMuMu_M_200_dbs0p04
2 200 0.04
SR2_BFFZprimeToMuMu_M_200_dbs0p04
1 500 1.0
SR1_BFFZprimeToMuMu_M_500_dbs1p0
2 500 1.0
SR2_BFFZprimeToMuMu_M_500_dbs1p0
1 200 0.5
SR1_BFFZprimeToMuMu_M_200_dbs0p5
2 200 0.5
SR2_BFFZprimeToMuMu_M_200_dbs0p5
1 175 0.04
SR1_BFFZprimeToMuMu_M_175_dbs0p04
2 175 0.04
SR2_BFFZprimeToMuMu_M_175_dbs0p04
1 350 0.04
SR1_BFFZprimeToMuMu_M_350_dbs0p04
2 350 0.04
SR2_BFFZprimeToMuMu_M_350_dbs0p04
1 500 0.5
SR1_BFFZprimeToMuMu_M_500_dbs0p5
2 500 0.5
SR2_BFFZprimeToMuMu_M_500_dbs0p5
1 125 0.04
SR1_BFFZprimeToMuMu_M_125_dbs0p04
2 125 0.04
SR2_BFFZprimeToMuMu_M_125_dbs0p04
1 200 1.0
SR1_BFFZprimeToMuMu_M_200_dbs1p0
2 200 1.0
SR2_BFFZprimeToMuMu_M_200_dbs1p0
1 350 1.0
SR1_BFFZprimeToMuMu_M_350_dbs1p0
2 350 1.0
SR2_BFFZprimeToMuMu_M_350_dbs1p0
1 350 0.5
SR1_BFFZprimeToMuMu_

In [40]:
era

'2016'